In [3]:



import os
import pandas as pd
from lxml import etree


ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}


def readfiles(orglist, pathtofiles):
    orgsFrame = pd.DataFrame()
    orgsDict = {}
    orgslist = []
    orgsref = []
    orgstext = []
    orgsvorkommen = []
    orgfiles = []
    orgslistvorkommen = []
    regestyear = []
    orgstype = []
    for filename in os.listdir(pathtofiles):
           regesttree = etree.parse(pathtofiles+"/"+filename)
           regestroot = regesttree.getroot()
           orgNodes = regestroot.xpath('.//*[contains(@ref,"org__") or contains(@corresp, "org__")]')
           date = createdate(regestroot.find(".//tei:creation/tei:date", ns))
           for rs in orgNodes:
               text = rs.xpath("normalize-space(.//text())")
               if 'ref' in rs.attrib:
                    ref = rs.attrib['ref'].replace("#","")
                    if ref not in orgsDict:
                        sublist = [text, 1, [filename], [date]]
                        orgsDict[ref] = sublist
                    else:
                        orgsDict[ref][1] += 1
                        orgsDict[ref][3].append(date)
                        if filename not in orgsDict[ref][2]:
                            orgsDict[ref][2].append(filename)
               elif 'corresp' in rs.attrib:
                    ref = rs.attrib['corresp'].replace("#","")
                    if ref not in orgsDict:
                        sublist = [text, 1, [filename], [date]]
                        orgsDict[ref] = sublist
                    else:
                        orgsDict[ref][1] += 1
                        orgsDict[ref][3].append(date)
                        if filename not in orgsDict[ref][2]:
                            orgsDict[ref][2].append(filename)
               else:
                   orgslist.append(text)
                   orgslistvorkommen.append(filename)


    for key, item in orgsDict.items():
        orgsref.append(key)
        orgstype.append(getorgtype(key, orglist))
        orgstext.append(item[0])
        orgsvorkommen.append(item[1])
        filestring = ", ".join(item[2])
        datestring = ", ".join(item[3])
        orgfiles.append(filestring)
        regestyear.append(datestring)


    for entry in orgslist:
        orgsref.append('')
        orgstext.append(entry)
        orgsvorkommen.append(1)

    for entry in orgslistvorkommen:
        orgfiles.append(entry)



    orgsFrame['ref_corresp'] = orgsref
    orgsFrame['type'] = orgstype
    orgsFrame['text'] = orgstext
    orgsFrame['anzahl'] = orgsvorkommen
    orgsFrame['xml'] = orgfiles
    orgsFrame['year'] = regestyear

    orgsFrame.to_csv("../data/output/csv/RefCorrespOrganisations.csv", sep="\t", encoding="utf-8")
    writer = pd.ExcelWriter('../data/output/xlsx/RefCorrespOrganisations.xlsx')
    orgsFrame.to_excel(writer, 'RefCorrespOrganisations', index=False)
    writer.save()


def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when']
        year = node.attrib['when'][:4]
        return year

def getorgtype(orgref, orglist):
    orgid = orgref.replace("#","")
    organisation = orglist.find(".//*[@xml:id='"+orgid+"']", ns)
    if organisation is not None:
        orgtype = organisation.attrib['type']
    else: orgtype = "Not in OrgList"
    return orgtype



orglist = etree.parse("../../data/lists/orgList.xml")
readfiles(orglist, "../../regests_QGW_PR_Stadt_ab_1400/done")
print("done")



[<Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd32a08>, <Element {http://www.tei-c.org/ns/1.0}rs at 0x7f7a0bd32b48>, <Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd32bc8>, <Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd32c08>]
[<Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd329c8>, <Element {http://www.tei-c.org/ns/1.0}rs at 0x7f7a0bd32848>, <Element {http://www.tei-c.org/ns/1.0}rs at 0x7f7a0bd36488>, <Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd36648>]
[<Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd366c8>, <Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd36788>, <Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd367c8>]
[<Element {http://www.tei-c.org/ns/1.0}roleName at 0x7f7a0bd32a08>, <Element {http://www.tei-c.org/ns/1.0}rs at 0x7f7a0bd36408>, <Element {http://www.tei-c.org/ns/1.0}rs at 0x7f7a0bd36e48>, <Element {http://www.tei-c.org/ns/1.0}rs at 0x7f7a0bd36ec8>, <Element {http://www.tei-c.